This code is designed to take the ASOS and AWOS station lists sourced from NCEI and the ASOS/AWOS station list sourced from ISD, compare and combine them. The code below is exploratory, designed to get a sense of how complete the merge is and what possible data cleaning issues exist. The complete code for performing the station merge is integrated into the ASOSAWOS_clean.py script, as the merge_station_list() function. The code below here is for reference only and should not be run.

In [ ]:
import pandas as pd
import boto3
from io import BytesIO
s3_cl = boto3.client('s3') # for lower-level processes

asosawos = s3_cl.get_object(Bucket='wecc-historical-wx', Key='1_raw_wx/ASOSAWOS/stationlist_ASOSAWOS.csv')
asosawos_list = pd.read_csv(BytesIO(asosawos['Body'].read()))  

asosawos_list # 399 records
    
isd = s3_cl.get_object(Bucket='wecc-historical-wx', Key='1_raw_wx/ASOSAWOS/stationlist_ISD_ASOSAWOS.csv')
isd_list = pd.read_csv(BytesIO(isd['Body'].read()))  

print(isd_list)

#print(asosawos_list.columns)
#print(isd_list.columns)

# Try match on WBAN, lat, lon
#print(asosawos_list[['WBAN', 'LAT', 'LON']])
#print(isd_list[['WBAN', 'LAT', 'LON']])

# Round asosawos down to 3 decimal points of accuracy
asosawos_round = asosawos_list.round({'LAT':3, 'LON':3})

# To do so, try exact match first
join = isd_list.merge(asosawos_round, left_on = ['WBAN', 'LAT', 'LON'], right_on = ['WBAN', 'LAT', 'LON'], how = 'inner')
join # 413 matches

missed = asosawos_list[~asosawos_list.NCDCID.isin(join.NCDCID)]
missed

    